In [19]:
# plotting stuff
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import cartopy.crs as crs

# built in python modules
import datetime
import os
import inspect
import sys

# python add-ons
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
from wrf import (to_np, getvar, ALL_TIMES, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)
import geocat.comp
import optwrf as ow
from optwrf import runwrf as rw

In [6]:
# Find the path to optwrf
optwrf_path = os.path.dirname(os.path.abspath(inspect.getfile(ow)))

/Users/swardy9230/Box Sync/01_Research/01_Renewable_Analysis/WRF_Solar_and_Wind/met4ene/optwrf/optwrf


In [13]:
in_yr = 2011
in_mo = 2
wrfdir = os.path.join(optwrf_path, 'data/')
eradir = os.path.join(optwrf_path, 'data/')

The wrfout file /Users/swardy9230/Box Sync/01_Research/01_Renewable_Analysis/WRF_Solar_and_Wind/met4ene/optwrf/optwrf/data/wrfout_processed_d01.nc does not exist. Check that your path.


In [20]:
def wrf_era5_regrid(in_yr, in_mo, wrfdir='./', eradir='/share/mzhang/jas983/wrf_data/data/ERA5/'):
    """

    :param in_yr:
    :param in_mo:
    :param wrfdir:
    :param eradir:
    :return:

    """
    # WRF file containing source grid
    wrffile = 'wrfout_processed_d01.nc'
    try:
        wrfdata = xr.open_dataset(wrfdir + wrffile)
    except FileNotFoundError:
        print(f'The wrfout file {wrfdir + wrffile} does not exist. Check that your path.')
        wrfdata = None
        eradata = None
        return wrfdata, eradata

    # Get wrf variable(s) to regrid
    wrf_lat = wrfdata.XLAT
    wrf_lon = wrfdata.XLONG

    # Read in and convert GHI from W m-2 to kW m-2
    ghi = wrfdata.ghi
    ghi = ghi/1000

    # Read in WPD, convert from W m-2 to kW m-2
    wpd = wrfdata.wpd
    wpd = wpd/1000

    # ERA data file(s)
    erafile = f'ERA5_EastUS_WPD-GHI_{str(in_yr).zfill(4)}-{str(in_mo).zfill(2)}.nc'
    try:
        eradata = xr.open_dataset(eradir + erafile)
    except FileNotFoundError:
        print(f'The wrfout file {eradir + erafile} does not exist. Check that your path.')
        eradata = None
        return wrfdata, eradata

    # Get variables to compare with regridded WRF variables.
    era_lat = eradata.latitude
    era_lon = eradata.longitude

    # Read in ERA_GHI, convert from W m-2 to kW m-2
    era_ghi = eradata.GHI
    era_ghi = era_ghi / 1000

    # Read in ERA_WPD, convert from W m-2 to kW m-2
    era_wpd = eradata.WPD
    era_wpd = era_wpd / 1000

    # Write these back to the xarray dataset
    eradata['ghi'] = era_ghi
    eradata['wpd'] = era_wpd

    # Do the regridding
    wrf_ghi_regrid = geocat.comp.rcm2rgrid(wrf_lat, wrf_lon, ghi, era_lat, era_lon)
    wrf_wpd_regrid = geocat.comp.rcm2rgrid(wrf_lat, wrf_lon, wpd, era_lat, era_lon)

    # Add the regridded variables to the WRF xarray dataset
    wrfdata['ghi_regrid'] = wrf_ghi_regrid
    wrfdata['wpd_regrid'] = wrf_wpd_regrid

    return wrfdata, eradata


In [21]:
wrfdata, eradata = wrf_era5_regrid(in_yr, in_mo, wrfdir, eradir)

AttributeError: module 'geocat.comp' has no attribute 'rcm2rgrid'

None


In [12]:
print(eradata)

None
